## Collecting Yelp Reviews using Scrapy

In [ ]:
#!pip install scrapy

Import necessary libraries

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

Create a spider class

In [2]:
## Create Spider

class yelp_review_spider(scrapy.Spider):
    name = 'yelp_review_spider'
    # start_requests method
    def start_requests(self):
        url = "https://www.yelp.ca/search?find_desc=vegan&find_loc=Vancouver%2C+BC&ns=1"
        yield scrapy.Request(url = url, callback = self.parse_front)
        
    # parse front page site
    def parse_front(self, response):
        resto_links = response.xpath("//a[contains(@class, 'css-1422juy')]/@href").extract()
        for link in resto_links:
            yield response.follow(url = link, callback = self.parse_pages)
            
    # parse other pages
    def parse_pages(self, response):
        resto_name = response.xpath("//h1[contains(@class, 'css-1x9iesk')]/text()")
        resto_name_ext = resto_name.extract_first().strip()
        review_p = response.css("p.comment__09f24__gu0rG.css-1sufhje")
        review_span = review_p.css("span.raw__09f24__T4Ezm")  
        review_list = review_span.extract()
        reviews_dict[resto_name_ext]['review'] = review_list
        
        #response.replace(body=response.body.replace(b'<br>', b'\n'))
        
        ## add authors, date published, rating
        '''
        author_div = response.xpath('//div[contains(@class, "user-")]')
        authors = author_div.css("a.css-1422juy::text")
        authors_list = authors.extract()
        reviews_dict[resto_name_ext]['author'] = [author for author in authors_list]
        
        dates = response.css("span.css-1e4fdj9::text")
        dates_list = dates.extract()
        reviews_dict[resto_name_ext]['date'] = [date for date in dates_list]
        
        ratings = response.css("//div[contains(@class, 'i-stars__09f24__foihJ.i-stars--regular')]/@aria-label")
        ratings_list = ratings.extract()
        reviews_dict[resto_name_ext]['rating'] = [rating for rating in ratings_list]
        '''
        
        

In [ ]:
html body yelp-react-root div div.margin-t3__09f24__riq4X.margin-b6__09f24__wgl48.border-color--default__09f24__NPAKY div.biz-details-page-container-outer__09f24__pZBzx.border-color--default__09f24__NPAKY div.biz-details-page-container-inner__09f24__L9S07.border-color--default__09f24__NPAKY div.margin-b6__09f24__wgl48.border-color--default__09f24__NPAKY div.stickySidebar--heightContext__09f24__O4ZFj.tableLayoutFixed__09f24__oM8X7.arrange__09f24__LDfbs.padding-b4__09f24__q6U6q.border--bottom__09f24___mg5X.border-color--default__09f24__NPAKY div.arrange-unit__09f24__rqHTg.arrange-unit-grid-column--8__09f24__HDezV.padding-r2__09f24__ByXi4.border-color--default__09f24__NPAKY div.border-color--default__09f24__NPAKY section.margin-t4__09f24__G0VVf.padding-t4__09f24__Y6aGL.border--top__09f24__exYYb.border-color--default__09f24__NPAKY div.css-79elbk.border-color--default__09f24__NPAKY div.border-color--default__09f24__NPAKY ul.undefined.list__09f24__ynIEd li.margin-b5__09f24__pTvws.border-color--default__09f24__NPAKY div.review__09f24__oHr9V.border-color--default__09f24__NPAKY div.margin-b2__09f24__CEMjT.border-color--default__09f24__NPAKY p.comment__09f24__gu0rG.css-1sufhje

In [3]:
from collections import defaultdict
reviews_dict = defaultdict(dict)

In [4]:
process = CrawlerProcess()
process.crawl(yelp_review_spider)
process.start()

2022-01-17 22:07:46 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-01-17 22:07:46 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2022-01-17 22:07:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-01-17 22:07:46 [scrapy.crawler] INFO: Overridden settings:
{}
2022-01-17 22:07:46 [scrapy.extensions.telnet] INFO: Telnet Password: 8b66e7870d458635
2022-01-17 22:07:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-01-17 22:07:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'sc

In [11]:
reviews_dict['MeeT in Gastown']['review'][-1]

'<span class=" raw__09f24__T4Ezm" lang="en">It starts with a mission. \xa0"MeeT\'s mission is to help transition the mainstream to eating more plant-based foods in a friendly, accepting and engaging venue" (from their website). \xa0Not only that, but they also aim to show people that comfort food that connects and nourishes us can be vegetarian/vegan, delicious, made from scratch, and can actually be quite affordable! \xa0Hands in the air for all those positive vibes!<br><br>My hubby and I (both vegetarian/plant-based, often vegan, and work in the health &amp; fitness industry) first discovered MeeT late summer 2019. \xa0We were in town for a half marathon, and after a long travel day, we wanted/needed something nutritious, fresh and delicious, and to dine in a place with a fun and inviting atmosphere. \xa0We pretty much hit the jackpot here at MeeT in Gastown!<br><br>Here\'s what we ordered and would recommend over and over again:<br>1. The Big Yum<br>2. Organic Quinoa &amp; Roasted Y